In [20]:

from gensim.models import Phrases
import pandas as pd
import numpy as np
import re


from gensim.models import Phrases

documents = ["the mayor of new york was there", "machine learning can be useful sometimes","new york mayor was present","machine learning is boring"]

sentence_stream = [doc.split(" ") for doc in documents]
print(sentence_stream)
bigram = Phrases(sentence_stream, min_count=1, threshold=2)
sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there',u'machine',u'learning']
print(bigram[sent])

[['the', 'mayor', 'of', 'new', 'york', 'was', 'there'], ['machine', 'learning', 'can', 'be', 'useful', 'sometimes'], ['new', 'york', 'mayor', 'was', 'present'], ['machine', 'learning', 'is', 'boring']]
['the', 'mayor', 'of', 'new_york', 'was', 'there', 'machine_learning']


In [26]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [w.lower()  for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            print(s)
            self.n_sent += 1
            return s
        except:
            return None
        

In [121]:
def buildSentences(text):
    sent = []
    tmp = []
    while "." in text:
        dot_index = text.index(".")
        sent.append(text[0:dot_index])
        text = text[dot_index+1:]
    for d in sent:
        _t = [s for s in d.split(" ") if len(s)!=0]
        tmp.append(_t)
    sent= tmp
    return sent
        
res = buildSentences("bonjour mon nom est cgw. je viens de dakar.")
##print(res)

In [215]:
import string
import re


def dropSpace(s,replace=""):
    return re.sub(r'[\t\r\n]',replace,s)

printable = set(string.printable)


skills = pd.read_csv("../data/UpdatedResumeDataSet.csv")
resume = skills["Resume"]
sentences_dataset  = []

for r in resume:
    r = ''.join(filter(lambda x: x in printable, r))
    r = dropSpace(r)
    r = r.lower()
    sentences_dataset.append(buildSentences(r))


In [221]:
tmp = []
for sentences in sentences_dataset:
    for s in sentences:
        tmp.append(s)

bigram = Phrases(tmp, min_count=5, threshold=2)
sent = [u'data', u'science', u'of', u'new', u'york', u'was', u'there',u'software',u'engineer',"big","data",u"computer","science","machine","learning"]
print(bigram[sent])


['data_science', 'of', 'new_york', 'was', 'there', 'software', 'engineer', 'big_data', 'computer_science', 'machine_learning']


In [209]:
bigram.save(u"../models/saved_model/phraser")

In [15]:
import pandas as pd
filepath = u"../data/resume_sentence_dataset.csv"

data = pd.read_csv(filepath)

data

,Sentence #,Word,Tag
0,0,Govardhana,Name
1,0,K,Name
2,0,Senior,Profil
3,0,Software,Profil
4,0,Engineer,Profil
...,...,...,...
75425,3018,Arc,O
75426,3018,"sight,",O
75427,3018,HP,O
75428,3018,Service,O


In [16]:
## try word embedding format word2vec
import multiprocessing
import gensim

from gensim.models import Word2Vec
import logging
from time import time

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = data.groupby("Sentence #").apply(lambda x : [w for w in x["Word"]])
type(sentences)
sentences = sentences.tolist()
print(len(sentences))

3018


In [27]:
cores = multiprocessing.cpu_count()
cores


4

In [18]:
w2v_model = Word2Vec(min_count=5,
                     window=4,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
w2v_model.corpus_count

2020-05-30 14:15:43,429 : INFO : collecting all words and their counts
2020-05-30 14:15:43,430 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-30 14:15:43,463 : INFO : collected 14729 word types from a corpus of 75430 raw words and 3018 sentences
2020-05-30 14:15:43,464 : INFO : Loading a fresh vocabulary
2020-05-30 14:15:43,479 : INFO : effective_min_count=5 retains 2279 unique words (15% of original 14729, drops 12450)
2020-05-30 14:15:43,481 : INFO : effective_min_count=5 leaves 56626 word corpus (75% of original 75430, drops 18804)
2020-05-30 14:15:43,498 : INFO : deleting the raw counts dictionary of 14729 items
2020-05-30 14:15:43,500 : INFO : sample=6e-05 downsamples 1204 most-common words
2020-05-30 14:15:43,502 : INFO : downsampling leaves estimated 22261 word corpus (39.3% of prior 56626)
2020-05-30 14:15:43,513 : INFO : estimated required memory for 2279 words and 300 dimensions: 6609100 bytes
2020-05-30 14:15:43,514 : INFO : resetting laye

Time to build vocab: 0.01 mins


3018

In [19]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

2020-05-30 14:15:48,376 : INFO : training model with 3 workers on 2279 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=4
2020-05-30 14:15:48,486 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-30 14:15:48,500 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-30 14:15:48,507 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-30 14:15:48,513 : INFO : EPOCH - 1 : training on 75430 raw words (22241 effective words) took 0.1s, 179164 effective words/s
2020-05-30 14:15:48,604 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-30 14:15:48,610 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-30 14:15:48,617 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-30 14:15:48,619 : INFO : EPOCH - 2 : training on 75430 raw words (22217 effective words) took 0.1s, 231016 effective words/s
2020-05-30 14:15:48,707 : INFO : worke

2020-05-30 14:15:51,377 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-30 14:15:51,404 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-30 14:15:51,427 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-30 14:15:51,429 : INFO : EPOCH - 21 : training on 75430 raw words (22307 effective words) took 0.2s, 129838 effective words/s
2020-05-30 14:15:51,527 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-30 14:15:51,543 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-30 14:15:51,555 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-30 14:15:51,557 : INFO : EPOCH - 22 : training on 75430 raw words (22230 effective words) took 0.1s, 199984 effective words/s
2020-05-30 14:15:51,644 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-30 14:15:51,659 : INFO : worker thread finished; awaiting finish of 1 more threads
2020

Time to train the model: 0.07 mins


In [28]:
import gensim
phraser = gensim.models.Phrases.load("../models/saved_model/phraser")
phraser

In [ ]:
w2v_model.save("saved_model/word2vecEmbedding")
word_vectors = w2v_model.wv
print("Number of word vectors: {}".format(len(word_vectors.vocab)))